# Notebook 5.2: Phylogenetics in practice

### Introduction

*Orthologs* are homologous gene copies that diverged from a common ancestral gene during speciation events. After speciation, lineages evolve independently such that orthologous gene copies accumulate different mutations over time. This process can continue through many speciation events. As mutations arise in the periods between speciation they leave behind a record of the existence of the common ancestral sequence. In this way, the mutational process constructs a record that we can eventually use to reconstruct the order and timing of speciation events. By comparing the orthologous gene sequences of the descendant lineages alive today we can contruct a phylogeny as a snapshot of the past. 

### Inferring phylogenies

In this notebook we will not dive too deep into the algorithmic and methodological complexity of phylogenetic inference. In its most basic form, the goal of molecular phylogenetics is to count the number of *character differences* (e.g., substitutions) between multiple orthologous sequences, and to build a tree that *minimizes* the number of steps needed to explain those differences. We can do this using simple parsimony methods (the fewest steps is the best), or with fancier methods that model DNA substitution as a probabilistic process (e.g., maximum likelihood).

The details of how various phylogenetic software tools differ, and what assumptions they make, is important to learn and relevant if you use phylogenetics for research. And we'll get to some of those details later. But for now, we'll focus on the *applied* end of things. How do I infer a phylogenetic tree in practice? We'll turn our attention back to the more theoretical and statistical details later. 

### Practical tree inference

There are three basic steps involved in inferring a phylogenetic tree. We'll complete a practical example from start to finish in this notebook. Then at the end your assignment will be to try to replicate this for an organism of your choice. 

1. Generate/retrieve sequence data from *orthologous gene regions*. For this we will use the NCBI API utilities that we learned about last time to query and fetch fasta files directly from a large database.   


2. Align the data to identify substitutions at *homologous sites*. For this we will use a command line program called **muscle**. There are many alignment programs available, I chose this one because it's easy to use. We will input a fasta file that includes all of our sequence data, but for which each ortholog may be a different length and may start or stop at a different position in the gene. The muscle program will ouput a new fasta file where the sequences are aligned by imputing "-" symbols to improve the alignment. We'll see an example of this below.


3. Infer a tree by entering the aligned data into a phylogenetic inference program. For this we'll enter the aligned fasta files into a program called **RAxML**. This is a commonly used phylogenetic inference program that uses Maximum Likelihood to infer the best tree to fit the data. As output RAxML produces a newick formatted tree file (like we learned about in the last notebook). Finally, we'll plot the tree result. 

In [71]:
import os
import subprocess
import toytree
import requests
import pandas as pd

<div class="alert alert-success">
    <b>Question [5]:</b> 
What are the 3 steps involved in practical phylogenetic inference? Answer concisely in Markdown below. 
</div>

## Retrieving sequence data

To find individual genes or gene regions sampled from particular species the best place to look is the NCBI Genbank sequence archive. This database includes both Sanger sequenced DNA regions that were targeted by researchers for sequencing using primers, as well as genes or gene regions from sequenced and annotated genomes. In last week's notebook we used API tools to access fasta data directly from the database. Here we'll use that code again, written as Python functions. Execute the cells to load these functions, and follow along by reading the code and comments in it. We'll mostly focus on the results that the code produces. 

#### Retrieve unique IDs for some number of samples
The `esearch` API returns unique IDs for the search term that we use. Our query below is searching for a particular *marker* that is commonly used in plant phylogenetic studies (`trnK`). This is a chloroplast gene that has an important function and so tends to be highly conserved, which makes it easy to sequence by designing primers to amplify it. However, it also contains an *intron* sequence nearby that can be sequenced along with it, and this region is more variable, and thus provides more phylogenetic information than the conserved part. It has historically been difficult to find genomic regions that are both easy to amplify and variable enough to work as good phylogenetic markers. The few regions that were first discovered to fit this category have been used extensively. And because of this inertia they have become what is called *universal markers* -- a few gene regions that have been sequenced across thousands of organisms by many research groups over several decades. These markers provide the best sampled phylogenetic data that we have, and through a community effort of collaboration and data sharing have made it possible to combine available data to build enormous phylogenies for thousands of species that help to elucidate the tree of life. 

In addition to targeting the `trnK` gene we're also restricting our query to a specific organism, the plant genus *Pedicularis*. This is a group of plants that I study. Please follow along using this search term for now, you will have a chance to explore other search terms later below. 



In [72]:
# execute this cell to create the get_uids function

def get_uids(term, retmax=10):
    "Search NCBI nucleotide database and return N sequence matches"
    
    # make a request to esearch 
    res = requests.get(
        url="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi", 
        params={
            "db": "nucleotide",
            "term": term,
            "sort": "length",
            "retmode": "text",
            "retmax": retmax,
            "tool": "genomics-course", 
            "email": "de2356@columbia.edu",
            },
        )
    
    # parse the xml output
    count = res.text.split("<Count>")[1].split("</Count>")[0].strip()
    
    # if nothing found then bail out
    if not int(count):
        raise ValueError("No UIDs found")
    
    # return the list of UIDs
    uids = []
    ids = res.text.split("<IdList>")[1].split("</IdList>")[0].strip()
    for item in ids.split("\n"):
        uids.append(item[4:-5])
    return uids

Our query statement to the database is stored as the variable `term` below. We enter this as an argument to the `get_uids()` function and tell it to return at most 20 results. The function returns a list of strings that represent unique IDs matching our query. 

In [73]:
# the search term
term = "trnk[GENE] AND Pedicularis[ORGN]"

# call the function to get uids and store as a list
ingroups = get_uids(term=term, retmax=20)

# show the first ten values
ingroups[:10]

['63253964',
 '63253962',
 '63253960',
 '63253958',
 '63253956',
 '63253954',
 '63253952',
 '63253950',
 '63253948',
 '63253946']

### Outgroup sample
As we learned in the last notebook, it is important to *root* your tree so that you know the direction of evolution when interpreting evolutionary relationships. One way to do this is to include an **outgroup** sample in your analysis -- a sample that you know is more distantly related to everything else. Here we'll include 2 samples from a closely related genus. To do this I changed the search term string below to look for the *same gene* but from a *different organism*. Finally we add the list results of this search (2 UIDs) with the list results of the first one (20 UIDs) to have a total of 22 samples in the final list `uids`. We could combine many searches to select samples from a variety of species, or genera, or families in this way. 

In [74]:
# the search term
term = "trnk[GENE] AND Orobanche[ORGN]"

# call the function to get uids and store as a list
outgroups = get_uids(term=term, retmax=2)

# show these uids
print(outgroups)

# add these uids to the list
uids = ingroups + outgroups

['1127299348', '6650062']


#### Retrieve fasta data for the UIDs

The next function fetches fasta data from the database based on the unique IDs that we got from the previous search. In the API call here we specify the strand of DNA, which will ensure they are all returned in the same orientation, and we also specify a start and stop limit just to ensure we do not get back sequences longer than 2000 base pairs. Here automation can sometimes be dangerous because there are errors in the metadata of the database. For example, people sometimes enter the wrong strand information into the metadata when they upload the sequences, which can cause your sequences to not align well. It is important to visually inspect your data to check for these types of errors, as we will do below. 


In [75]:
# execute this cell to create the get_fasta function

def get_fasta(uids):
    """
    Fetch fasta sequence data from NCBI for a list of uids
    and return as a dictionary of {name: sequence}.
    """
    # make a request to efetch 
    res = requests.get(
        url="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi", 
            params={
            "db": "nucleotide",
            "id": ",".join(uids),
            "seq_start": "1",
            "seq_stop": "2000",
            "strand": "1",
            "retmode": "text",   
            "rettype": "fasta",
            "tool": "genomics-course", 
            "email": "de2356@columbia.edu",
            },
        )
    
    # split fasta string into separate fasta sequences
    fastas = res.text.strip().split("\n\n")
    
    # write to output file
    with open("sequences.fasta", 'w') as out:
        for fasta in fastas:

            # separate name and sequence
            name, sequence = fasta.split("\n", 1)

            # reorder and shorten fasta names for easier reading
            bits = name.split()
            genus = bits[1][0]
            species = bits[2]
            accession = bits[0][1:].split(":")[0]
            name = ">{}_{}_{}".format(genus, species, accession)

            # remove line breaks from sequence
            sequence = sequence.replace("\n", "")

            # write to file
            out.write("{}\n{}\n".format(name, sequence))
    
    # print statement
    print("Wrote {} sequences to ./sequences.fasta".format(len(fastas)))
    

In [76]:
# run the get_fasta command
fastadict = get_fasta(uids)

Wrote 22 sequences to ./sequences.fasta


#### Looking at the fasta file

We now downloaded orthologous gene sequences for 22 samples (20 *Pedicularis* and 2 *Castilleja*). The function above printed a statement saying that it printed the data to `./sequences.fasta`. You can open the file in your file viewer to look at it. Or, we can also peek at it here in the notebook. Let's look at just the first few columns and rows of data. Here I'm going to use a cool trick in Pandas to display the DNA as a DataFrame with colored cells. 

In [77]:
# execute to create the following functions

def highlight_dna(cell):
    "A function to color cells by DNA base"
    color = {"A": 'red', "T": 'blue', "C": 'green', "G": 'yellow', "-": "lightgrey"}
    return 'background-color: {}'.format(color[cell])



def colored_slice(fasta, start, stop):
    "returns a colored dataframe over a given slice"
    # load seq data 
    with open(fasta) as infile:

        # load sequence file as a dictionary
        fdict = {}
        for fa in infile.read().strip()[1:].split("\n>"):
            name, seq = fa.split("\n")
            fdict[name] = list(seq)[start: stop]

        # make dataframe from dictionary
        df = pd.DataFrame(fdict, index=range(start, stop)).T

    # show dataframe as colored cells
    return df.style.applymap(highlight_dna)


We can now use this function to look at a slice of this fasta file. You can scroll to the right to see more columns as well. Each column here represents one sequenced base. When all of the samples have the same base at a position this means that the site is not variable (at least given our sampling). No mutations have occurred at that site since these species diverged from each other. You can see that most columns for the Pedicularis samples are invariant, but then there is a lot of mismatch with the two outgroup samples in the bottom two rows.  

Now remember, the sequences in this file have not been aligned yet, so we expect that they may look pretty bad at this stage. As you scroll to the right you'll see that the ingroup samples (Pedicularis) also fall out of phase pretty quickly. That is because there was an insertion or deletion at some point which is not being accounted for. Let's see if aligning makes this look better. 

In [78]:
# we can't tell yet that the sequences are not aligned
colored_slice("./sequences.fasta", 0, 100)

## Alignment

The muscle alignment program is a command line program that can be called as `muscle -in infile -out outfile`. In the function below we use the Python library `subprocess` to call and run the program from within Python with `subprocess.call()`. This is a convenient way in which to use Python code as "glue" to tie together programs written in other languages with tasks you want to complete in Python. 

The program muscle uses an algorithm to insert "-" characters into the alignment so that it minimizes differences among sites. The details of this algorithm are interesting, and there are many ways to perform alignments, but that is outside the scope of this tutorial. Below we simply call the muscle program and parse the output to write it back to a file in a way that will be easy for us to load it back in as a pandas DataFrame to look at again with color. 

In [79]:
# execute this cell to create the align_fasta function

def align_fasta(fasta_file):
    
    # the output aligned file name to use
    aligned = fasta_file + ".aligned"
    
    # run muscle alignment program on fasta file to produce output file
    cmd = ["muscle", "-in", fasta_file, "-out", aligned]
    subprocess.call(cmd)

    # read in results of aligned file
    with open(aligned) as infile:
        fastas = infile.read().strip()[1:].split("\n>")
        
    # remove newlines from aligned file
    with open(aligned, 'w') as out:
        for fasta in fastas:

            # separate name and sequence
            name, sequence = fasta.split("\n", 1)

            # remove line breaks from sequence
            sequence = sequence.replace("\n", "")

            # write to file
            out.write(">{}\n{}\n".format(name, sequence))
    
    # print statement
    print("Wrote aligned file to {}".format(aligned))
    

In [80]:
# run the alignment program
align_fasta("./sequences.fasta")

Wrote aligned file to ./sequences.fasta.aligned


#### Look at the aligned fasta file
Muscle produced a new fasta file where the data is now aligned. Let's take a look to ensure that it looks better than our previous non-aligned data. Scroll around and see that there are now grey "-" characters imputed to improve the alignment. Is likely not perfect, but hopefully it's pretty accurate. Now when there is variation at a site we expect that variation represents substitutions that have happened at a single orthologous site. 

In [81]:
# looking in the middle of the file it's clear they are not aligned
colored_slice("./sequences.fasta.aligned", 100, 200)


## Infer a phylogeny

Similarly to how we called the program `muscle` above, here we will call the program `raxmlHPC` using the Python subprocess library again. The raxml software is complex and requires a number of arguments. You do not need to worry about those for now. You can look at the documentation for the software to learn more. The take home message here is that in just a minute or two this software can take our input aligned fasta file and infer a phylogeny as well as an estimate of confidence/support in that estimate. We'll discuss more later the complexities of how tree inference works.

In [82]:
# execute this cell to create the infer_tree command

def infer_tree(aligned_fasta):
    
    # remove previous results if they exist
    oldname = "RAxML_info.{}".format(os.path.basename(aligned_fasta))
    if os.path.exists(oldname):
        os.remove(oldname)
        
    # the raxml command
    cmd = [
        "raxmlHPC", 
        "-f", "a",
        "-s", aligned_fasta, 
        "-m", "GTRGAMMA", 
        "-n", os.path.basename(aligned_fasta),
        "-p", "12345", 
        "-x", "54321",
        "-w", os.path.realpath("."),
        "-N", "100",
    ]
        
    # print the raxml command
    print("running: {}\n".format(" ".join(cmd)))
    subprocess.call(cmd)
    
    # print statement
    print("Tree file written to ./RAxML_bipartitions.{}".format(
        os.path.basename(aligned_fasta)))
    

In [83]:
# run the tree inference 
infer_tree("./sequences.fasta.aligned")

running: raxmlHPC -f a -s ./sequences.fasta.aligned -m GTRGAMMA -n sequences.fasta.aligned -p 12345 -x 54321 -w /home/deren/Documents/genomics-course/5-phylogenetics/notebooks -N 100

Tree file written to ./RAxML_bipartitions.sequences.fasta.aligned


### Plot/draw the phylogeny

Now that you've inferred a tree the next step is of course to check your results by visualization. Plot the tree. We can then compare the topology to our expectation based on morphology, or geography, or other expectations. Do all of the branch lengths look reasonable? If not, perhaps you have a bad sequence in the alignment. Visually inspect it and consider removing it. 

In the tree below everything looks reasonable and likely correct given my knowledge of this group of organisms. I rooted the tree on the two outgroup sequences using the `.root()` function and a *wildcard* selector that will match any names starting with "O_". In the final tree I also included a scalebar. You'll notice that on this tree the tips (black lines) do not all line up at 0 (the dashed lines are added just make the name labels align to make reading it easier). This is because this is not a time-scaled tree. The branch lengths are in units of substitutions per site. Some tips branches had more substitutions than others, which may reflect a higher rate of molecular evolution. Using this phylogeny we can now begin to test other evolutionary hypotheses. 

In [84]:
# load the tree
tre = toytree.tree("./RAxML_bestTree.sequences.fasta.aligned")

# root on the outgroups
rtre = tre.root(regex="O_*")

# draw with support values
rtre.draw(width=500, tip_labels_align=True, scalebar=True);

P_verticillata_AY949769.1 P_verticillata_AY949762.1 P_szetschuanica_AY949754.1 P_ternata_AY949758.1 P_semitorta_AY949756.1 P_anas_AY949764.1 P_rupicola_AY949752.1 P_integrifolia_AY949763.1 P_streptorhyncha_AY949753.1 P_thamnophila_AY949761.1 P_rhinanthoides_AY949755.1 P_schistostegia_AY949767.1 P_rudis_AY949751.1 P_trichocymba_AY949760.1 P_bella_AY949757.1 P_furfuracea_AY949765.1 P_axillaris_AY949768.1 P_cranolopha_AY949759.1 P_bifida_AY949750.1 P_davidii_AY949766.1 O_hederae_AF051995.1 O_boninsimae_KU647699.1 0.00 0.02 0.04 0.06 0.08

### Challenge: Use these functions to infer a tree for some other organismal group

We've now written functions to automate the minimal three steps required for inferring a phylogeny. And we can now reuse these functions to perform these tasks again with a different starting query to the database. 


<div class="alert alert-success">
    <b>Action [6]:</b> 
Infer a phylogeny for a group of organisms of your choice, and using a genetic marker of your choice, by following the instructions below. 
</div>

#### 1. Find data
Every organism is not in the NCBI database for all genes. The hardest part of this challenge will be finding a good genetic marker that is sequenced across enough taxa in the group you are searching. I would recommend starting by searching the internet for the name of common phylogenetic markers used in your organism (or in animals versus plants). You can search these terms directly on NCBI [https://www.ncbi.nlm.nih.gov/nuccore/](https://www.ncbi.nlm.nih.gov/nuccore/). Search for terms like "universal phylogenetic marker", or do a google scholar search for "phylogeny" + the name of the organism you're interested in. Find a phylogenetic study that used a marker and try to sample that data yourself here.

Once you develop a good search term then you can insert it into the code below. If you get stuck head to the chat room. Don't try to move on to the next step until you get some UIDs printed below. If your search term does not match anything the function below will raise an error telling you that. 

After you get your ingroup samples you next need to find an outgroup. This means finding an organism outside of your core group, but still pretty closely related. This will require you to do some research as well. NCBI includes a lot of information about taxonomy, as do many other sites including Wikipedia, or published phylogenetic papers. 

In [85]:
# call the function to get uids and store as a list
term = "trnK[GENE] AND Pedicularis[ORGN]"
ingroup = get_uids(term=term, retmax=20)

# call the function to uids from an *outgroup* and add to list
oterm = "trnK[GENE] AND Orobanchaceae[ORGN]"
outgroup = get_uids(term=oterm, retmax=2)

# show uids
uids = ingroup + outgroup
print(uids)

['63253964', '63253962', '63253960', '63253958', '63253956', '63253954', '63253952', '63253950', '63253948', '63253946', '63253944', '63253942', '63253940', '63253938', '63253936', '63253934', '63253932', '63253930', '63253928', '63253926', '887515644', '887515611']


In [86]:
# get fasta data for the UIDs
get_fasta(uids)

Wrote 22 sequences to ./sequences.fasta


#### 2. Align the fasta file
Call the alignment function on your fasta file and then use the `colored_slice` function to look at your alignment. Does it look good, or a complete mess? If it's a mess do not proceed to the next step. Maybe you need to try a different search term, a more restrictive definition of the gene segment you are searching for. 

In [87]:
# run the alignment program
align_fasta("./sequences.fasta")

# looking in the middle of the file it's clear they are not aligned
colored_slice("./sequences.fasta.aligned", 300, 350)

Wrote aligned file to ./sequences.fasta.aligned


,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349
N_pedicularoides_KP940486.1,A,A,A,A,T,G,A,A,A,A,A,C,C,A,A,A,A,G,G,T,C,G,T,C,T,T,A,T,T,C,A,A,A,A,C,C,C,C,A,A,T,T,A,T,G,T,T,A,T,G
N_pedicularoides_KP940492.1,-,-,-,-,-,A,A,A,A,C,T,G,G,G,G,C,A,T,G,T,C,G,T,C,C,A,A,A,T,C,A,T,C,G,-,-,-,-,-,-,-,-,-,-,G,T,C,C,G,G
P_thamnophila_AY949761.1,-,-,-,-,-,G,A,A,A,A,A,A,T,A,G,C,A,T,G,T,C,G,T,A,T,C,A,A,T,A,G,A,G,A,-,-,-,-,-,-,-,-,-,-,G,T,T,C,T,G
P_streptorhyncha_AY949753.1,-,-,-,-,-,G,A,A,A,A,A,A,T,A,G,C,A,T,G,T,C,G,T,A,T,C,A,A,T,A,G,A,G,A,-,-,-,-,-,-,-,-,-,-,G,T,T,C,T,G
P_bella_AY949757.1,-,-,-,-,-,G,A,A,A,A,A,A,T,A,G,C,A,T,G,T,C,G,T,A,T,C,A,A,T,A,G,A,G,A,-,-,-,-,-,-,-,-,-,-,G,T,T,C,T,G
P_verticillata_AY949769.1,-,-,-,-,-,G,A,A,A,A,A,A,T,A,G,C,A,T,G,T,C,G,T,A,T,C,A,A,T,A,G,A,G,A,-,-,-,-,-,-,-,-,-,-,G,T,T,C,T,G
P_verticillata_AY949762.1,-,-,-,-,-,G,A,A,A,A,A,A,T,A,G,C,A,T,G,T,C,G,T,A,T,C,A,A,T,A,G,A,G,A,-,-,-,-,-,-,-,-,-,-,G,T,T,C,T,G
P_szetschuanica_AY949754.1,-,-,-,-,-,G,A,A,A,A,A,A,T,A,G,C,A,T,G,T,C,G,T,A,T,C,A,A,T,A,G,A,G,A,-,-,-,-,-,-,-,-,-,-,G,T,T,C,T,G
P_semitorta_AY949756.1,-,-,-,-,-,G,A,A,A,A,A,A,T,A,G,C,A,T,G,T,C,G,T,A,T,C,A,A,T,A,G,A,G,A,-,-,-,-,-,-,-,-,-,-,G,T,T,C,T,G
P_ternata_AY949758.1,-,-,-,-,-,G,A,A,A,A,A,A,T,A,G,C,A,T,G,T,C,G,T,A,T,C,A,A,T,A,G,A,G,A,-,-,-,-,-,-,-,-,-,-,G,T,T,C,T,G


### Infer the tree
If your alignment is good then there's not much to this step. Run it and wait for it to finish. 




In [ ]:
# run the tree inference 
infer_tree("./sequences.fasta.aligned")

### Draw your tree

Before drawing your tree be sure to change the code below to select the proper names of your outgroup sequences to root the tree since their names are likely different than the ones that I used. If you need help with rooting the tree you can find more information [here](https://toytree.readthedocs.io/en/latest/5-toytrees.html#Rooting-trees). 

In [88]:
# load the tree
tre = toytree.tree("./RAxML_bestTree.sequences.fasta.aligned")

# root on the outgroups
rtre = tre.root(regex="O_*")

# draw with support values
rtre.draw(width=500, tip_labels_align=True, scalebar=True);

P_verticillata_AY949769.1 P_verticillata_AY949762.1 P_szetschuanica_AY949754.1 P_ternata_AY949758.1 P_semitorta_AY949756.1 P_anas_AY949764.1 P_rupicola_AY949752.1 P_integrifolia_AY949763.1 P_streptorhyncha_AY949753.1 P_thamnophila_AY949761.1 P_rhinanthoides_AY949755.1 P_schistostegia_AY949767.1 P_rudis_AY949751.1 P_trichocymba_AY949760.1 P_bella_AY949757.1 P_furfuracea_AY949765.1 P_axillaris_AY949768.1 P_cranolopha_AY949759.1 P_bifida_AY949750.1 P_davidii_AY949766.1 O_hederae_AF051995.1 O_boninsimae_KU647699.1 0.00 0.02 0.04 0.06 0.08


<div class="alert alert-success">
    <b>Question [7]:</b> 
Reflect on your phylogeny above. Do you think it is accurate? Can you find a published phylogeny online for this group of organisms? If so, include it in your answer. What gene/marker did they use in that study? Do your trees show the same relationships for the selected organisms?
    
</div>


<div class="alert alert-success">
    <b>Question [8]:</b>     
If you were writing a program to automate fetching data from online and inferring a tree, what would you do differently? What steps could be made easier? How do you imagine this will change as more and more annotated genes from assembled genomes are added to NCBI?
</div>